# Electricity Consumption Prediction (ECP)

In this project we're going to predict electricity consumption of each user for future periods.

Here is some info and stats about the data:
* **Number of unique users: 124675**
* **Total number of records: 5601193**

Also you can find `headers.xslx` file in `dataset` directory to find some more descriptions about the header names or features.

In [69]:
# Import required packages

import datetime
import pandas as pd
from persiantools.jdatetime import JalaliDate

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

## 1.1. Peek at Our Data

There is no substitute for looking at the raw data.

Looking at the raw data can reveal insights that you cannot get any other way. It can also plant seeds that may later grow into ideas on how to better preprocess and handle the data for machine learning tasks.

We can review the first 5 rows of our data here.

In [70]:
# Load the dataset
df = pd.read_csv("../dataset/dbBills.csv")

# Print top 5 reeocrds of dataset
df.head()

,xSubscriptionId_fk,xIdentityNo,xOmorCode,xBakhshCode,xCycleCode,xMamorCode,xRegionName,xIsTropical,xUsageGroupName,xFamilyNum,xTariffOldCode,xFaze,xAmper,xCounterBuldingNo,xTimeControlCode,xBillStartDate,xBillEndDate,xMeduimKw,xHighKw,xLowKw
0,9397665,2/6/15/01/25/10/7651,15,1,25,10,شهری,0,عمومي,1,2990,3,25,12606909,1,1391/08/25,1391/09/09,538,0,0
1,9396214,2/6/15/01/44/10/4321,15,1,44,10,شهری,0,عمومي,1,2990,3,25,8336853341,1,1391/08/18,1391/09/09,1999,0,0
2,9396214,2/6/15/01/44/10/4321,15,1,44,10,شهری,0,عمومي,1,2990,3,25,8336853341,1,1391/09/09,1391/10/03,0,0,0
3,8952093,2/6/15/04/32/04/4870,15,4,32,4,روستایی,0,عمومي,1,2990,3,25,37337459,1,1386/11/27,1387/02/05,1065,0,0
4,8952093,2/6/15/04/32/04/4870,15,4,32,4,روستایی,0,عمومي,1,2990,3,25,37337459,1,1387/02/05,1387/04/05,560,0,0


We can see that the first column lists the row number, which is handy for referencing a specific observation.

## 1.2. Dimensions of Our Data

We must have a very good handle on how much data we have, both in terms of rows and columns.

Too many rows and algorithms may take too long to train. Too few and perhaps you do not have enough data to train the algorithms.
Too many features and some algorithms can be distracted or suffer poor performance due to the curse of dimensionality.
You can review the shape and size of your dataset by printing the shape property.

In [71]:
# Dimensions of your data
print("Our data has dimention of {} Records and {} Columns or Features.".format(df.shape[0], df.shape[1]))

Our data has dimention of 5601193 Records and 20 Columns or Features.


## 1.3. Data Type For Each Attribute

The type of each attribute is important.

Strings may need to be converted to floating point values or integers to represent categorical or ordinal values.

We can get an idea of the types of attributes by peeking at the raw data, as above. We can also list the data types used by the DataFrame to characterize each attribute using the dtypes property.

In [72]:
# Data Types for Each Attribute
types = df.dtypes
types.head(20)

xSubscriptionId_fk     int64
xIdentityNo           object
xOmorCode              int64
xBakhshCode            int64
xCycleCode             int64
xMamorCode             int64
xRegionName           object
xIsTropical            int64
xUsageGroupName       object
xFamilyNum             int64
xTariffOldCode         int64
xFaze                  int64
xAmper                 int64
xCounterBuldingNo      int64
xTimeControlCode       int64
xBillStartDate        object
xBillEndDate          object
xMeduimKw              int64
xHighKw                int64
xLowKw                 int64
dtype: object

We can see that most of the attribute types are objects (categorical string values).

## 1.4. Data Cleaning and Transformation

For apply more effective analysis on data we need to reshape our data. So we transform categorical data to numbers using one-hot encoding and fill **NaN** valuses with average amount as a way for data cleaning.

In [73]:
# Utils

def days_difference(start_date, end_date):    
    start_date = start_date.split("/")
    end_date = end_date.split("/")
    
    jalali_start_date = JalaliDate(int(start_date[0]), int(start_date[1]), int(start_date[2]))
    jalali_end_date = JalaliDate(int(end_date[0]), int(end_date[1]), int(end_date[2]))

    difference = jalali_end_date.to_gregorian() - jalali_start_date.to_gregorian()
    
    return difference.days

def extract_month(input_date):
    return int(input_date.split("/")[1])

In [74]:
# Clean Dataset

df = df.drop('xIsTropical', 1) # Since it's a constant value
df = df.drop('xOmorCode', 1) # Since it's a constant value
df = df.drop('xCounterBuldingNo', 1) # Since we can use 'xTimeControlCode' field
df = df.drop('xIdentityNo', 1) # Since we can use other decomposed fields.
df = df.drop('xMamorCode', 1) # It seems there is no useful information in this field

# Map Persian strings to English strings for field: 'xRegionName'
region_maps = {'شهری': 'Shahri', 'روستایی': 'Roustaei'}
df['xRegionName'].replace(region_maps, inplace=True)
# One-Hot Encoding on categorical values
# use pd.concat to join the new columns with your original dataframe
df = pd.concat([df,pd.get_dummies(df['xRegionName'], prefix='xRegionName')],axis=1)
df = df.drop('xRegionName', 1) # Since we can use one-hot encoded features instead

# Map Persian strings to English strings for field: 'xUsageGroupName'
usage_group_maps = {'عمومي': 'Omoomi',
                    'خانگي': 'Khanegi',
                    'كشاورزي': 'Keshavarzi',
                    'ساير مصارف': 'Sayer',
                    'صنعتي': 'Sanati'}
df['xUsageGroupName'].replace(usage_group_maps, inplace=True)
# One-Hot Encoding on categorical values
# use pd.concat to join the new columns with your original dataframe
df = pd.concat([df,pd.get_dummies(df['xUsageGroupName'], prefix='xUsageGroupName')],axis=1)
df = df.drop('xUsageGroupName', 1) # Since we can use one-hot encoded features instead

# One-Hot Encoding on categorical values
# use pd.concat to join the new columns with your original dataframe
df = pd.concat([df,pd.get_dummies(df['xBakhshCode'], prefix='xBakhshCode')],axis=1)
df = df.drop('xBakhshCode', 1) # Since we can use one-hot encoded features instead

# One-Hot Encoding on categorical values
# use pd.concat to join the new columns with your original dataframe
df = pd.concat([df,pd.get_dummies(df['xTimeControlCode'], prefix='xTimeControlCode')],axis=1)
df = df.drop('xTimeControlCode', 1) # Since we can use one-hot encoded features instead

# Adding new features using 'xBillStartDate' and 'xBillEndDate'.
df['days_difference'] = df.apply (lambda row: days_difference(row.xBillStartDate, row.xBillEndDate), axis=1)
df['month'] = df.apply (lambda row: extract_month(row.xBillStartDate), axis=1)

df = df.drop('xBillStartDate', 1) # Since we've extracted useful parts before
df = df.drop('xBillEndDate', 1) # Since we've extracted useful parts before

# Move target or labels to the end of dataset
xMeduimKw_tmp = df.pop('xMeduimKw') # remove column xMeduimKw and store it in xMeduimKw_tmp
xHighKw_tmp = df.pop('xHighKw') # remove column xHighKw and store it in xHighKw_tmp
xLowKw_tmp = df.pop('xLowKw') # remove column xLowKw and store it in xLowKw_tmp

df['xMeduimKw']=xMeduimKw_tmp # add xMeduimKw series as a 'new' column.
df['xHighKw']=xHighKw_tmp # add xMeduimKw series as a 'new' column.
df['xLowKw']=xLowKw_tmp # add xMeduimKw series as a 'new' column.

df.head()

# xCycleCode ???
# xTariffOldCode ???

# Save cleaned dataset for using in later analysis
# df.to_csv('../dataset/dbBills_cleaned.csv')

,xSubscriptionId_fk,xCycleCode,xFamilyNum,xTariffOldCode,xFaze,xAmper,xRegionName_Roustaei,xRegionName_Shahri,xUsageGroupName_Keshavarzi,xUsageGroupName_Khanegi,xUsageGroupName_Omoomi,xUsageGroupName_Sanati,xUsageGroupName_Sayer,xBakhshCode_1,xBakhshCode_2,xBakhshCode_4,xTimeControlCode_1,xTimeControlCode_2,xTimeControlCode_3,days_difference,month,xMeduimKw,xHighKw,xLowKw
0,9397665,25,1,2990,3,25,0,1,0,0,1,0,0,1,0,0,1,0,0,14,8,538,0,0
1,9396214,44,1,2990,3,25,0,1,0,0,1,0,0,1,0,0,1,0,0,21,8,1999,0,0
2,9396214,44,1,2990,3,25,0,1,0,0,1,0,0,1,0,0,1,0,0,24,9,0,0,0
3,8952093,32,1,2990,3,25,1,0,0,0,1,0,0,0,0,1,1,0,0,68,11,1065,0,0
4,8952093,32,1,2990,3,25,1,0,0,0,1,0,0,0,0,1,1,0,0,62,2,560,0,0
